# vLLM inference performance tests

In [ ]:
pip install --upgrade vllm

In [2]:
import importlib.metadata
importlib.metadata.version("torch")

'2.1.2'

In [3]:
importlib.metadata.version("vllm")

'0.4.0.post1'

In [4]:
importlib.metadata.version("xformers")

'0.0.23.post1'

In [5]:
importlib.metadata.version("flash_attn")

'2.5.6'

## vLLM performance test code

In [1]:
# model = "meta-llama/Meta-Llama-3-8B-Instruct"
# model = "casperhansen/llama-3-8b-instruct-awq"
model = "TechxGenus/Meta-Llama-3-8B-Instruct-GPTQ"

max_model_len=8192

In [2]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Pass the default decoding hyperparameters of Qwen1.5-7B-Chat
# max_tokens is for the maximum length for generation.
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=512)

# llm = LLM(model, kv_cache_dtype="fp8_e5m2", gpu_memory_utilization=0.99, max_model_len=max_model_len)
# llm = LLM(model, quantization="awq", kv_cache_dtype="fp8_e5m2", gpu_memory_utilization=0.99, max_model_len=max_model_len)
llm = LLM(model, quantization="gptq", kv_cache_dtype="fp8_e5m2", gpu_memory_utilization=0.99, max_model_len=max_model_len)

#llm = LLM(model, quantization="awq", kv_cache_dtype="fp8_e5m2", gpu_memory_utilization=0.99)
#llm = LLM(model, quantization="gptq", kv_cache_dtype="fp8_e5m2", max_model_len=max_model_len)
#llm = LLM(model, quantization="gptq", max_model_len=max_model_len)
#llm = LLM(model, quantization="awq", max_model_len=max_model_len)
#llm = LLM(model, quantization="awq", kv_cache_dtype="fp8_e5m2", max_model_len=max_model_len)

/workspace/wordslab-llms/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 04-21 17:08:52 config.py:211] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 04-21 17:08:52 utils.py:256] Not found nvcc in /usr/local/cuda. Skip cuda version check!
INFO 04-21 17:08:52 config.py:381] Using fp8_e5m2 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop. Currently we only support fp8 without scaling factors and make e5m2 as a default format.
INFO 04-21 17:08:52 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='TechxGenus/Meta-Llama-3-8B-Instruct-GPTQ', tokenizer='TechxGenus/Meta-Llama-3-8B-Instruct-GPTQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=gptq, enforce_eager=False, kv_cache_dtype=fp8_e5m2, device_config=cud

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 04-21 17:08:53 utils.py:357] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 04-21 17:08:53 selector.py:16] Using FlashAttention backend.
INFO 04-21 17:08:54 weight_utils.py:177] Using model weights format ['*.safetensors']


/workspace/wordslab-llms/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 5735.72 MB. The target location /models/huggingface/hub only has 0.00 MB free disk space.
  warnings.warn(
/workspace/wordslab-llms/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 5735.72 MB. The target location /models/huggingface/hub/models--TechxGenus--Meta-Llama-3-8B-Instruct-GPTQ/blobs only has 0.00 MB free disk space.
  warnings.warn(


model.safetensors:   0%|          | 0.00/5.74G [00:00<?, ?B/s]

INFO 04-21 17:13:39 model_runner.py:104] Loading model weights took 5.3472 GB
INFO 04-21 17:13:41 gpu_executor.py:94] # GPU blocks: 16851, # CPU blocks: 4096
INFO 04-21 17:13:41 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-21 17:13:41 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-21 17:13:45 model_runner.py:867] Graph capturing finished in 5 secs.


In [3]:
import time

In [49]:
prompt = "Quels sont les avantages du Crédit Mutuel ?"

# System prompt
messages = [
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": prompt}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Agricole ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Société Générale ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la BNP ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Banque populaire ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Caisse d'épargne ?"}
]
]

# Generate outputs
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

start_time = time.time()  # Record the start time
outputs = llm.generate(text, sampling_params)
end_time = time.time()  # Record the end time
    
# Print the outputs.
tokenscount = 0
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    tokenscount = tokenscount + len(output.outputs[0].token_ids)
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    
print(f"Performance: {int(tokenscount/(end_time-start_time))} tokens/sec")

Processed prompts: 100%|██████████| 6/6 [00:06<00:00,  1.13s/it]

Prompt: '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont les avantages du Crédit Mutuel ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n', Generated text: "Crédit Mutuel est une banque coopérative française qui offre de nombreux avantages à ses clients. Voici quelques-uns des principaux avantages du Crédit Mutuel :\n\n1. **Intérêts plus faibles** : Crédit Mutuel propose des taux d'intérêt plus bas que les banques traditionnelles pour ses comptes courants, épargne et prêt.\n2. **Frais de transactions réduits** : Crédit Mutuel ne prévoit pas de frais de transactions pour les opérations réalisées sur son réseau de distributeurs.\n3. **Comptes courants sans frais** : Les comptes courants Crédit Mutuel sont gratuits, sans frais d'opérations ni de maintenance.\n4. **Prêt à 0%** : Crédit Mutuel propose des prêts à 0% d'intérêt po

## vLLM performance tests results

Max tokens = 512

- "meta-llama/Meta-Llama-3-8B-Instruct": 52 tokens/sec (batch size 1)

- "casperhansen/llama-3-8b-instruct-awq": 112 tokens/sec (batch size 1)

- "TechxGenus/Meta-Llama-3-8B-Instruct-GPTQ"
  - 125 tokens/sec (batch size 1)
  - 240 tokens/sec (batch size 2)
  - 342 tokens/sec (batch size 3)
  - 448 tokens/sec (batch size 4)
  - 510 tokens/sec (batch size 5)
  - 600 tokens/sec (batch size 6) 
  - 612 tokens/sec (batch size 12)
  - 565 tokens/sec (batch size 10)
  - 543 tokens/sec (batch size 8)

- model = "Qwen/Qwen1.5-32B-Chat-GPTQ-Int4"
- max_model_len=10576
- quantization="gptq"
- kv_cache_dtype="fp8_e5m2"
- gpu_memory_utilization=0.99

42 tokens / sec

- model = "Qwen/Qwen1.5-14B-Chat-GPTQ-Int4"
- quantization="gptq"
- kv_cache_dtype="fp8_e5m2"
- max_model_len=22624

80 tokens/sec

- model = "Qwen/Qwen1.5-14B-Chat-GPTQ-Int4"
- quantization="gptq"
- max_model_len=11744

80 tokens/sec

- model = "Qwen/Qwen1.5-14B-Chat-GPTQ-Int8"
- quantization="gptq"
- kv_cache_dtype="fp8_e5m2"
- max_model_len=4016
- gpu_memory_utilization=1

50 tokens/sec

- model = "Qwen/Qwen1.5-14B-Chat-AWQ"
- quantization="awq"
- max_model_len=10544

74 tokens/sec

- model = "Qwen/Qwen1.5-14B-Chat-AWQ"
- quantization="awq"
- kv_cache_dtype="fp8_e5m2"
- max_model_len=21104

74 tokens/sec



- model = "NousResearch/Hermes-2-Pro-Mistral-7B"
- max_model_len=12544
- gpu_memory_utilization=0.95

55 tokens/sec

- solidrust/Nous-Hermes-2-Mistral-7B-DPO-AWQ
- quantization="awq"
- kv_cache_dtype="fp8_e5m2"
- gpu_memory_utilization=0.99
- (max_seq_len=32768)

125 tokens/sec